# 워드투벡터
- https://wikidocs.net/22660

### cmd 명령어
- https://dumps.wikimedia.org/kowiki/latest/ 에서 덤프파일 다운로드 (kowiki-latest-pages-articles.xml.bz2)
- https://github.com/attardi/wikiextractor 에서 zip파일 다운로드한 뒤 덤프bz2파일과 동일한 경로에 풀기
- cmd에서 경로 찾아간 뒤 압축해제 명령어 <br>
(python WikiExtractor.py kowiki-latest-pages-articles.xml.bz2 => 약 10분정도 걸림 

### 추출된 text데이터 합치기
- copy AA디렉토리의 경로\wiki* wikiAA.txt
>copy AB디렉토리의 경로\wiki* wikiAB.txt <br>
copy AC디렉토리의 경로\wiki* wikiAC.txt <br>
copy AD디렉토리의 경로\wiki* wikiAD.txt <br>
copy AE디렉토리의 경로\wiki* wikiAE.txt <br>
copy AF디렉토리의 경로\wiki* wikiAF.txt
- copy 현재 디렉토리의 경로\wikiA* wiki_data.txt

# txt데이터 읽어오기
- 직접하기엔 kowiki 전처리 + mecab ko dic부분 처리가 곤란
- 18년 7월에 올라온 데이터 임포트 https://github.com/roboreport/doc2vec-api
- 혼자 직접해보려면 https://www.lucypark.kr/courses/2015-dm/text-mining.html 참고

In [1]:
import sys 
import codecs

with open('wiki_data.txt', 'r', encoding = 'utf-8') as f:
    lines = f.readlines()

In [8]:
str_lines = str(lines)

In [7]:
str_lines

['<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">\n',
 '지미 카터\n',
 '\n',
 '제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.\n',
 '\n',
 '지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n',
 '\n',
 '1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n',
 '\n',
 '1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.\n',
 '\n',
 '카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n',
 '카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다.\n',
 '\n',
 '그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 1979년 백악관에서 양국 간의 평화조약으로 이끌어졌다. 또한 소련과 제2차 전략 무기 제한 협상에 조인했다.\n',
 '\n',
 '카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.\n',
 '\n',


In [6]:
from konlpy.tag import Okt; t = Okt()
tokens_ko = t.morphs(lines)

RuntimeError: No matching overloads found for tokenize in find. at native\common\jp_method.cpp:127

In [62]:
import pandas as pd
import numpy as np
import tensorflow as tf
from bs4 import BeautifulSoup
import ipdb

from eunjeon import Mecab  

from gensim.models import Word2Vec

mecab = Mecab()

In [65]:
learning_rate = 0.001
dim_embed = 200
n_epochs = 20
window_size = 5
min_count = 3

#wiki_file = 'wiki_data.txt'
#with open( wiki_file ) as f:
#    wiki_contents = f.read()
with open('wiki_data.txt', 'r', encoding = 'utf-8') as f:
    wiki_contents = f.readlines()
    wiki_docs = map(lambda x: filter(lambda y: y != '', x.text.split('\n')), BeautifulSoup( wiki_contents ).find_all('doc'))
    wiki_paragraphs = [item for sublist in wiki_docs for item in sublist]

paragraph_list = []
for wiki_paragraph in wiki_paragraphs:
    wiki_paragraph_pos = map(lambda x: x[0] + '^/'+ x[1], mecab.pos( wiki_paragraph ))
    if len(wiki_paragraph_pos) > 2:
        paragraph_list.append( wiki_paragraph_pos )

del wiki_paragraphs
word2vec_model = Word2Vec( size=dim_embed, alpha=learning_rate, min_count=min_count, workers=-1 )
word2vec_model.build_vocab( paragraph_list )

for epoch in range( n_epochs ):
    print("Training Epoch:", epoch)
    word2vec_model.train( paragraph_list )
    word2vec_model.alpha *= 0.99

word2vec_model.save('word2vec_ex_model')

C:\Users\sbg02\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\sbg02\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


TypeError: expected string or bytes-like object

# 토큰화

In [54]:
from konlpy.tag import Twitter
twitter = Twitter()

C:\Users\sbg02\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [56]:
a = '내 똥이 너 똥보다 더 굵다'

In [ ]:
twitter.morphs(a)

# gensim으로 학습시키기
- 파라미터는 일단 wikidocs설명 그대로
- https://wikidocs.net/22660

In [52]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=lines, size=100, window=5, min_count=5, workers=4, sg=0)
model.save('word2vec_model')

KeyboardInterrupt: 

In [43]:
self.wv.most_similar('영화', topn=30)

NameError: name 'self' is not defined

In [ ]:
# 일단 이거 먼저 돌리기

In [73]:
import sys
import time
import glob
import unicodedata
from eunjeon import Mecab
from gensim.models import Word2Vec

# 모델의 파라미터들입니다.
WINDOW=5
EMBEDDING_SIZE = 200
BATCH_SIZE = 1024
ITER = 10

def read_text(fin):
    # 전처리된 위키백과 파일을 읽어 들입니다.
    corpus_li = []
    mecab = Mecab()
    for line in open(fin):
        # 깨지는 글자를 처리하기 위해 unicodedata.normalize 함수를 이용해
        # NFKC로변환합니다.
        line = unicodedata.normalize('NFKC', line)
        try:
            # 첫 글자가 숫자로 시작하는 문장을 말뭉치에 추가합니다.
            _ = int(line[0])
            corpus_li.append(' '.join(mecab.nouns(line)) + '\n')

        except ValueError:
            # 첫 글자가 한글로 시작하는 문장을 말뭉치에 추가합니다.
            if ord(line[0]) >= ord('가') and ord(line[0]) <= ord('힇'):
                corpus_li.append(' '.join(mecab.nouns(line))+'\n')
            else:
                pass
    print('# of lines in corpus',len(corpus_li))
    return(corpus_li)

def train_word2vec(corpus_li, fout_model):
    # read_text에서 생성한 말뭉치를 이용해 word2vec을 학습시킵니다.
    model = Word2Vec(corpus_li, sg=1, size=EMBEDDING_SIZE, window=WINDOW, min_count=5, workers=8, batch_words=BATCH_SIZE, iter=ITER)
    model.init_sims(replace=True) #clean up memory
    model.save(fout_model)
    return(model)

# 전처리된 파일을 한번에 읽어 들이기 위한 정규식
'''
input_pattern = 'wiki_data.txt'
fin_li = glob.glob(input_pattern)

for fin in fin_li:
    corpus_li = read_text(fin)
'''
with open('wiki_data.txt', 'r', encoding = 'utf-8') as f:
    corpus_li = f.readlines()

# 모델학습
model = train_word2vec(corpus_li, 'test_model2222.txt')
print(model.most_similar('프랑스', topn=20))
print(model.most_similar(positive=['한국','파리'], negative=['서울']))

KeyboardInterrupt: 

# 뭔가 잘못된거같아서 두번째 학습

In [47]:
# 모델의 파라미터들입니다.
WINDOW = 5
EMBEDDING_SIZE = 200
BATCH_SIZE = 10000
ITER = 10

def train_word2vec(lines, fout_model):
    # read_text에서 생성한 말뭉치를 이용해 word2vec을 학습시킵니다.
    model = Word2Vec(lines, sg=1, size=EMBEDDING_SIZE, window=WINDOW, min_count=5, workers=3, batch_words=BATCH_SIZE, iter=ITER)
    model.init_sims(replace=True) #clean up memory
    model.save(fout_model)
    return(model)

In [48]:
# 모델학습
model = train_word2vec(lines, 'test_model_second.txt')
print(model.most_similar('프랑스', topn = 20))
print(model.most_similar(positive = ['한국','파리'], negative = ['서울']))

C:\Users\sbg02\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: "word '프랑스' not in vocabulary"

# 4번째 시도
- http://blog.theeluwin.kr/post/146591096133/%ED%95%9C%EA%B5%AD%EC%96%B4-word2vec

In [9]:
from konlpy.tag import Okt
okt = Okt()

okt.pos("오버워치")
# [('오버', 'NNG'), ('워', 'UN'), ('하', 'XSV'), ('지', 'ECD')]

[('오버', 'Noun'), ('워치', 'Noun')]

In [10]:
import codecs

corpus = codecs.open('corpus.txt', 'w', encoding='utf-8')

def flat(content):
    return ["{}/{}".format(word, tag) for word, tag in okt.pos(content)]

corpus.write(' '.join(flat("메이드복 입은 제이미 귀엽다")) + '\n')
# 메이드/Noun 복/Noun 입은/Verb 제이/Noun 미/Adjective 귀엽/Adjective 다/Eomi

In [13]:
import gensim

sentences = [["my", "name", "is", "jamie"], ["jamie", "is", "cute"]]
model = gensim.models.Word2Vec(sentences)

RuntimeError: you must first build vocabulary before training the model

In [14]:
class SentenceReader:

    def __init__(self, filepath):
        self.filepath = filepath

    def __iter__(self):
        for line in codecs.open(self.filepath, encoding='utf-8'):
            yield line.split(' ')

In [18]:
sentences_vocab = SentenceReader('wiki_data.txt')
sentences_train = SentenceReader('wiki_data.txt')

import multiprocessing

config = {
    'min_count': 5,  # 등장 횟수가 5 이하인 단어는 무시
    'size': 300,  # 300차원짜리 벡터스페이스에 embedding
    'sg': 1,  # 0이면 CBOW, 1이면 skip-gram을 사용한다
    'batch_words': 10000,  # 사전을 구축할때 한번에 읽을 단어 수
    'iter': 10,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수
    'workers': multiprocessing.cpu_count(),
}

model = gensim.models.Word2Vec(**config)
model.build_vocab(sentences_vocab)
model.train(sentences_train)

ValueError: You must specify either total_examples or total_words, for proper job parameters updationand progress calculations. The usual value is total_examples=model.corpus_count.